<a href="https://colab.research.google.com/github/MichalMaczek/praca_inz/blob/main/Clustering.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1.

In [27]:
import pandas as pd
import numpy as np
import os
import glob
import matplotlib.pyplot as plt

from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [28]:
XY = pd.read_csv('/content/drive/MyDrive/Inż/Adaptive/XY_512_150_adap.csv', sep=',', index_col=0)

lista = []
for (columnName, columnData) in XY.iteritems():
  lista.append(columnName)

X = pd.DataFrame(XY, columns= lista[:-1])
Y = pd.DataFrame(XY, columns = ["Label"])

In [29]:
TRAINING = [3, 4, 5, 7, 10, 11, 12, 13]
TEST = [1, 2, 6, 8, 9]
training_names = ['CTU_' + str(TRAINING[j]) + '_' for j in range(len(TRAINING))]
test_names = ['CTU_' + str(TEST[j]) + '_' for j in range(len(TEST))]

In [30]:
xy_training = XY[XY.index.str.startswith(tuple(training_names))]
x_train = pd.DataFrame(xy_training, columns= lista[:-1])
y_train = pd.DataFrame(xy_training, columns = ["Label"])

xy_test = XY[XY.index.str.startswith(tuple(test_names))]
x_test = pd.DataFrame(xy_test, columns= lista[:-1])
y_test = pd.DataFrame(xy_test, columns = ["Label"])

In [31]:
#standaryzacja danych trenujących
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
scaled_features = scaler.fit_transform(x_train)
x_train_scaled = pd.DataFrame(scaled_features, index=x_train.index)

In [32]:
from sklearn.cluster import DBSCAN, OPTICS
from sklearn.preprocessing import StandardScaler

#klasa dla klastrów
class Cluster:
    bot = False
    hosts = []
    coord = []
    
    def __init__(self, bot, hosts, coord):
        self.bot = bot
        self.hosts = hosts
        self.coord = coord

# Number of bins: 512

## Threshold = 150

### Model

In [33]:
#liczba binów oraz zbiór EPS
no_of_bins = 512
multiplier = [0.01 * n for n in range(1, 10)]
EPS = [ round(x * no_of_bins,1) for x in multiplier]

Learning

In [34]:
#hiperparametry algorytmu DBSCAN
#parametr algorithm - nie pomaga w ogóle
algorithm = ['auto', 'ball_tree', 'kd_tree', 'brute', 'auto']
metric = ['cityblock', 'euclidean']

In [35]:
#grupowanie
test, labels = [{} for i in range(2)]
for eps in EPS:
  dbscan = DBSCAN(eps=eps, min_samples=1,  metric='cityblock').fit(x_train_scaled)
  labels[eps] = dbscan.labels_

  clusters = []
  for label in range(max(labels[eps]) + 1):
    hosts = []
    is_bot = False
    for id, clus in enumerate(labels[eps]):
      if label == clus:
        hosts.append(list(x_train.index)[id])

        if xy_training['Label'].loc[x_train.index[int(id)]] == 1:
          is_bot = True

    coords = [x_train.loc[host] for host in hosts]

    #obliczenie pozycji klastra
    cluster_position = np.mean(coords, axis=0)
    clusters.append(Cluster(is_bot, hosts, cluster_position))
    test[eps] = clusters
  print(max(labels[eps]))

589
550
514
482
451
421
386
358
338


EVALUATION

In [36]:
import tensorflow as tf
#klasyfikacja
precision, recall, f1_score, accuracy = [{} for i in range(4)]
for eps in EPS:
  tp, fp, tn, fn = [0 for i in range(4)]
  
  for k in range(len(x_test)):

    distances = {}

    for cluster in test[eps]:
      #obliczenie odległości próbki od klastrów
      distances[cluster] = tf.norm(np.array(cluster.coord) - np.array(x_test.loc[x_test.index[k]]), ord=1)
    #znalezienie najbliższego klastra dla próbki
    closest_cluster = min(distances, key=distances.get)
    
    if closest_cluster.bot:
      if xy_test['Label'].loc[x_test.index[k]] == 1:
        tp += 1
      else:
        fp += 1
        
    else:
      if xy_test['Label'].loc[x_test.index[k]] == 1:
        fn +=1
      else:
        tn += 1

  precision[eps] = float(tp) / (tp + fp) if (tp + fp) != 0 else 0
  recall[eps] = float(tp) / (tp + fn) if (tp + fn) != 0 else 0
  f1_score[eps] = 2 * (recall[eps] * precision[eps]) / (recall[eps] + precision[eps]) if (recall[eps] + precision[eps]) != 0 else 0
  accuracy[eps] = float(tp + tn) / (tp + tn + fp + fn) if (tp + tn + fp + fn) != 0 else 0

  print(str(eps) + ": [" + str(tp) + " , " + str(fn) + "\n \t" + str(fp) + " , " + str(tn) + "]")

5.1: [12 , 2
 	5 , 707]
10.2: [12 , 2
 	0 , 712]
15.4: [12 , 2
 	0 , 712]
20.5: [13 , 1
 	0 , 712]
25.6: [13 , 1
 	0 , 712]
30.7: [13 , 1
 	0 , 712]
35.8: [13 , 1
 	0 , 712]
41.0: [13 , 1
 	2 , 710]
46.1: [13 , 1
 	2 , 710]


METRICS

In [37]:
#wypisanie metryk
new_labs = {}
for eps in EPS:
  print("EPSILON: " + str(eps))
  print("Precision: " + str(precision[eps]))
  print("Recall: " + str(recall[eps]))
  print("Accuracy: " + str(accuracy[eps]))
  print("F1 score: " + str(f1_score[eps]))
  print("No of clusters: " + str(max(labels[eps])))
  new_labs[eps] = max(labels[eps])
  print("-----------------------------------------------------")

EPSILON: 5.1
Precision: 0.7058823529411765
Recall: 0.8571428571428571
Accuracy: 0.990358126721763
F1 score: 0.7741935483870968
No of clusters: 589
-----------------------------------------------------
EPSILON: 10.2
Precision: 1.0
Recall: 0.8571428571428571
Accuracy: 0.9972451790633609
F1 score: 0.923076923076923
No of clusters: 550
-----------------------------------------------------
EPSILON: 15.4
Precision: 1.0
Recall: 0.8571428571428571
Accuracy: 0.9972451790633609
F1 score: 0.923076923076923
No of clusters: 514
-----------------------------------------------------
EPSILON: 20.5
Precision: 1.0
Recall: 0.9285714285714286
Accuracy: 0.9986225895316805
F1 score: 0.962962962962963
No of clusters: 482
-----------------------------------------------------
EPSILON: 25.6
Precision: 1.0
Recall: 0.9285714285714286
Accuracy: 0.9986225895316805
F1 score: 0.962962962962963
No of clusters: 451
-----------------------------------------------------
EPSILON: 30.7
Precision: 1.0
Recall: 0.928571428571

In [ ]:
#zapisanie parametrów do wykresów
np.savetxt("/content/drive/MyDrive/Inż/Plots/precision_1024_150_reg_0-1.csv", np.asarray(list(precision.values())), delimiter=",")
np.savetxt("/content/drive/MyDrive/Inż/Plots/recall_1024_150_reg_0-1.csv", np.asarray(list(recall.values())), delimiter=",")
np.savetxt("/content/drive/MyDrive/Inż/Plots/accuracy_1024_150_reg_0-1.csv", np.asarray(list(accuracy.values())), delimiter=",")
np.savetxt("/content/drive/MyDrive/Inż/Plots/f1_score_1024_150_reg_0-1.csv", np.asarray(list(f1_score.values())), delimiter=",")
np.savetxt("/content/drive/MyDrive/Inż/Plots/no_of_clusters_1024_150_reg_0-1.csv", np.asarray(list(new_labs.values())), delimiter=",")